# Setup

Before you can begin using the Transcoder API, run the following command in Cloud Shell to enable the API:

In [1]:
!gcloud services enable transcoder.googleapis.com

Get the project id (to be used in your application):

In [2]:
!echo $(gcloud config get-value core/project)

truster-tester-program


Install IPython and the Translation API client library:

In [3]:
!pip install google-cloud-video-transcoder

In [4]:
import argparse

from google.cloud.video import transcoder_v1
from google.cloud.video.transcoder_v1.services.transcoder_service import (
    TranscoderServiceClient,
)

In [5]:
PROJECT_ID = 'truster-tester-program'

The PROJECT_ID variable will be used for requests in the next steps.

# Upload a video to your Cloud Storage bucket

# Create a transcoding job

By default, the Transcoder API transcodes video using H.264 encoding and MP4, HLS, and MPEG-DASH packaging. For each input video, it delivers output renditions in high definition (1280x720 pixels) and standard definition (640x360 pixels).

In [6]:
def create_job_from_preset(
    project_id: str,
    location: str,
    input_uri: str,
    output_uri: str,
    preset: str,
) -> transcoder_v1.types.resources.Job:
    """Creates a job based on a job preset.

    Args:
        project_id: The GCP project ID.
        location: The location to start the job in.
        input_uri: Uri of the video in the Cloud Storage bucket.
        output_uri: Uri of the video output folder in the Cloud Storage bucket.
        preset: The preset template (for example, 'preset/web-hd').

    Returns:
        The job resource.
    """

    client = TranscoderServiceClient()

    parent = f"projects/{project_id}/locations/{location}"
    job = transcoder_v1.types.Job()
    job.input_uri = input_uri
    job.output_uri = output_uri
    job.template_id = preset

    response = client.create_job(parent=parent, job=job)
    print(f"Job: {response.name}")
    return response

In [7]:
create_job_from_preset(
    project_id=PROJECT_ID,
    location='europe-west1',
    input_uri='gs://pjs-munich-datathon-bucket/input/dreamfall.mov',
    output_uri='gs://pjs-munich-datathon-bucket/output/',
    preset='preset/web-hd'
)

Job: projects/104534638680/locations/europe-west1/jobs/851ba39d-3c60-4859-a205-9afb6e9742b7


name: "projects/104534638680/locations/europe-west1/jobs/851ba39d-3c60-4859-a205-9afb6e9742b7"
config {
  inputs {
    key: "input0"
    uri: "gs://pjs-munich-datathon-bucket/input/dreamfall.mov"
  }
  edit_list {
    key: "atom0"
    inputs: "input0"
    start_time_offset {
    }
  }
  elementary_streams {
    video_stream {
      h264 {
        width_pixels: 640
        height_pixels: 360
        frame_rate: 30.0
        bitrate_bps: 550000
        pixel_format: "yuv420p"
        rate_control_mode: "vbr"
        crf_level: 21
        gop_duration {
          seconds: 3
        }
        vbv_size_bits: 550000
        vbv_fullness_bits: 495000
        entropy_coder: "cabac"
        b_frame_count: 3
        aq_strength: 1.0
        profile: "high"
        preset: "veryfast"
      }
    }
    key: "video-stream0"
  }
  elementary_streams {
    video_stream {
      h264 {
        width_pixels: 1280
        height_pixels: 720
        frame_rate: 30.0
        bitrate_bps: 2500000
        pi

# Check the status of your transcoding job

In [8]:
def get_job_state(
    project_id: str,
    location: str,
    job_id: str,
) -> transcoder_v1.types.resources.Job:
    """Gets a job's state.

    Args:
        project_id: The GCP project ID.
        location: The location this job is in.
        job_id: The job ID.

    Returns:
        The job resource.
    """

    client = TranscoderServiceClient()

    name = f"projects/{project_id}/locations/{location}/jobs/{job_id}"
    response = client.get_job(name=name)

    print(f"Job state: {str(response.state.name)}")
    return response


In [11]:
get_job_state(
    project_id=PROJECT_ID,
    location='europe-west1',
    job_id='851ba39d-3c60-4859-a205-9afb6e9742b7'
)

Job state: SUCCEEDED


name: "projects/104534638680/locations/europe-west1/jobs/851ba39d-3c60-4859-a205-9afb6e9742b7"
config {
  inputs {
    key: "input0"
    uri: "gs://pjs-munich-datathon-bucket/input/dreamfall.mov"
  }
  edit_list {
    key: "atom0"
    inputs: "input0"
    end_time_offset {
      seconds: 265
      nanos: 233333000
    }
    start_time_offset {
    }
  }
  elementary_streams {
    video_stream {
      h264 {
        width_pixels: 640
        height_pixels: 360
        frame_rate: 30.0
        bitrate_bps: 550000
        pixel_format: "yuv420p"
        rate_control_mode: "vbr"
        crf_level: 21
        gop_duration {
          seconds: 3
        }
        vbv_size_bits: 550000
        vbv_fullness_bits: 495000
        entropy_coder: "cabac"
        b_frame_count: 3
        aq_strength: 1.0
        profile: "high"
        preset: "veryfast"
      }
    }
    key: "video-stream0"
  }
  elementary_streams {
    video_stream {
      h264 {
        width_pixels: 1280
        height_pixels

# Play your video